In [4]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import warnings
from operator import itemgetter
import yaml
import os
import yaml
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
# from src.vector_store_handler import VectorStoreHandler
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
# from langchain_core.parsers import StrOutputParser

In [5]:
# Suppress all warnings
warnings.filterwarnings('ignore')

# Example usage
def example_function():
    warnings.warn("This is a warning message", UserWarning)

example_function()  # This warning will not be displayed

In [6]:
class DocumentRetriever:
    def __init__(self, persist_directory):
        self.persist_directory = persist_directory
        self.embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={'device': 'mps'})
        self.vector_db = Chroma(persist_directory=self.persist_directory, embedding_function=self.embeddings)

    def retrieve_documents(self, query, top_k=5):
        # Retrieve top_k relevant document chunks based on the query
        return self.vector_db.similarity_search(query, k=top_k)

In [7]:


# class DocumentRetriever:
#     def __init__(self, persist_directory):
#         self.persist_directory = persist_directory
#         self.embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={'device': 'mps'})
#         self.vector_db = Chroma(persist_directory=self.persist_directory, embedding_function=self.embeddings)
#         self.retriever = self.vector_db.as_retriever()

#     def retrieve_documents(self, query, top_k=5):
#         # Retrieve top_k relevant document chunks based on the query
#         return self.retriever.similarity_search(query, k=top_k)

In [8]:
class RAGChatBot:
    def __init__(self, retriever, openai_api_key):
        self.retriever = retriever
        os.environ["OPENAI_API_KEY"] = openai_api_key
        self.chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
        self.prompt_template = hub.pull("rlm/rag-prompt")

    def format_docs(self, docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def generate_response(self, user_query):
        # Retrieve documents relevant to the user query
        relevant_docs = self.retriever.retrieve_documents(user_query)
        relevant_context = self.format_docs(relevant_docs)
        # for relevant_doc in relevant_docs:
            # print(relevant_doc.metadata['source'])
            # print(relevant_doc.page_content)

        user_query = user_query + " Is the patient appropriate for inpatient or outpatient surgery? Why?"
#         instructions = """
#         1. You are a highly knowledgeable and empathetic medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
#         2. If a user chats about some topic on medical device, but doesn't ask/chat about inpatient or outpatient surgery, gently ask the user if you should help him regarding the inpatient vs outpatient surgery
#         3. If a user asks questions outside the medical domain, gently remind them that your expertise is focused on healthcare and surgical advice.
#         4. Chat with the user using the given context. If the context is not at all relevant to the question asked. Tell your reasoning behind your answer and let the user know that you are not sure of the answer, due to lack of relevent context on your end to answer the question.
#         5. If you feel that the data provided is insufficient to answer the question, gently ask the user about more information.
#   """
        instructions = """1. You are a highly knowledgeable medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context given. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save huge costs if you can correctly identify the case of Outpatient Surgery, so please keep this in mind."""
        # Construct the input data for the chain
        input_data = {
            "context": relevant_context,
            "question": user_query,
            "system": instructions# Directly use the user query here
            # "GUIDELINES": instructions
        }
        
        # Create a prompt using the template
        formatted_prompt = self.prompt_template.format(**input_data)
        
        # Generate a response using the LLM with invoke method
        response = self.chat_model.invoke(formatted_prompt)
        
        return response.content

    def generate_response_no_rag(self, question):
        template = """You are a highly knowledgeable and empathetic medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.

Guidelines:
1. **Focus on Medical Context:** Always center your responses around medical topics, particularly those related to surgical procedures and patient care settings.
2. **Inquire When Necessary:** If you require more information to make an informed decision, politely ask the user specific questions about the surgery details, such as the type of surgery, patient health status, or any existing medical conditions.
3. **Educational Approach:** Explain your suggestions clearly, using simple language that both healthcare professionals and patients can understand. Provide insights into the benefits and considerations of each option.
4. **Boundary Setting:** If a user asks questions outside the medical domain, gently remind them that your expertise is focused on healthcare and surgical advice.

Example Interaction:
- **User Query:** "I need to know if my knee surgery should be inpatient or outpatient."
- **Response:** "Knee surgeries can often be performed as outpatient procedures if the patient is in good health and the surgery is minimally invasive. However, if there are underlying health concerns or if a more complex procedure is needed, an inpatient setting might be safer. Could you provide more details about your specific situation?"

By adhering to these guidelines, you ensure that every interaction is informative, supportive, and within the scope of medical advisement.
        """
        template1 = """1. You are a highly knowledgeable medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context you have. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save huge costs if you can correctly identify the case of Outpatient Surgery, so please keep this in mind."""
        system_message_prompt = SystemMessagePromptTemplate.from_template(template1)
        human_template = "{text}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        msg = self.chat_model.invoke(
            chat_prompt.format_prompt(
                text=question,
            ).to_messages()
        )

        return msg.content



    def generate_answer_with_chat_context(self, messages):
        """
        Generates a response with the last few messages and retrieved context using the chat model.

        Args:
            messages (list): List of dictionaries representing the conversation history.

        Returns:
            str: The generated response.
        """
        # Extract the last 4 messages plus the current one
        recent_messages = messages[-5:]  # Last 5 messages (4 historical + current)

        # Prepare input for retrieval (use only the content of messages, excluding the current one)
        retrieval_input = "\n".join(
            f"{message['role'].capitalize()}: {message['content']}" for message in recent_messages[:-1]
        )

        # Attempt to retrieve external documents
        try:
            retrieved_docs = self.retriever.retrieve_documents(retrieval_input)
            retrieved_context = self.format_docs(retrieved_docs)
        except Exception as e:
            retrieved_context = "No additional information could be retrieved."
            print(f"Error during document retrieval: {e}")

        # Ensure retrieved context is a valid string
        retrieved_context = str(retrieved_context)

        # Add a system message with the retrieved context
        system_message_prompt = """1. You are a highly knowledgeable medical chat assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context you have. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save HUGE costs if you can correctly identify the case of Outpatient Surgery, so need not be conservative if you are confident, with the context you have.
        3. If the question/chat is outside the medical domain, gently remind your expertise."""
        system_message = SystemMessage(content= system_message_prompt + f"\n **RELEVANT INFORMATION RETREIVED**:\n{retrieved_context}")
        # system_message = SystemMessage(content="You are a Medical QA chat assistant and you are specialized in making decisions between Inpatient vs Outpatient surgeries.")

        # Convert recent conversation into LangChain message objects
        formatted_recent_messages = []
        for message in recent_messages:
            if message["role"] == "user":
                formatted_recent_messages.append(HumanMessage(content=message["content"]))
            elif message["role"] == "ai":
                formatted_recent_messages.append(AIMessage(content=message["content"]))
            else:
                raise ValueError(f"Unexpected role: {message['role']}")

        # Combine system message and formatted recent messages
        input_messages = [system_message] + formatted_recent_messages
        #formatted_recent_messages.append(HumanMessage(content="Outpatient Surgery saves a lot of money. So never be conservative until otherwise inpatient surgery is ABSOULETELY necessary."))
        # input_messages =  formatted_recent_messages
        # print(input_messages)

        # Generate the response using the chat model
        try:
            llm_response = self.chat_model.invoke(input_messages)
        except Exception as e:
            print(f"Error during model generation: {e}")
            raise

        # Parse the response if needed
        parsed_response = StrOutputParser().parse(llm_response)

        return parsed_response.content


    # def generate_answer_with_chat_context(self, messages):
    #     """
    #     Generates a response with the last few messages and retrieved context using the chat model.

    #     Args:
    #         messages (list): List of dictionaries representing the conversation history.

    #     Returns:
    #         str: The generated response.
    #     """
    #     # Extract the last 4 messages plus the current one
    #     recent_messages = messages[-5:]  # Last 5 messages (4 historical + current)

    #     # Prepare input for retrieval (use only the content of messages, excluding the current one)
    #     retrieval_input = "\n".join(
    #         message['content'] for message in recent_messages[:-1]
    #     )

    #     # Attempt to retrieve external documents
    #     try:
    #         retrieved_docs = self.retriever.retrieve_documents(retrieval_input)
    #         retrieved_context = self.format_docs(retrieved_docs)
    #     except Exception as e:
    #         retrieved_context = "No additional information could be retrieved."
    #         print(f"Error during document retrieval: {e}")

    #     # Ensure retrieved context is a valid string
    #     retrieved_context = str(retrieved_context)

    #     # Instructions for the assistant
    #     instructions = """1. You are a highly knowledgeable medical chat assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate to be conducted as an inpatient or outpatient procedure based on the context you have. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
    # 2. You can save huge costs if you can correctly identify the case of Outpatient Surgery, so need not be conservative if you are confident, with the context you have.
    # 3. If the question/chat is outside the medical domain, gently remind your expertise.
    # 4. If you are not sure, either ask questions to user or say you need more information or that you don't know."""

    #     # Construct input data for the LLM
    #     input_data = {
    #         "context": retrieved_context,
    #         "question": recent_messages[-1]['content'],  # Use the latest user query
    #         "GUIDELINES": instructions
    #     }
        
    #     # Create a prompt using your template
    #     formatted_prompt = self.prompt_template.format(**input_data)

    #     # Generate a response using the LLM with invoke method
    #     try:
    #         response = self.chat_model.invoke(formatted_prompt)
    #         parsed_response = StrOutputParser().parse(response)
    #     except Exception as e:
    #         print(f"Error during model generation: {e}")
    #         parsed_response = "An error occurred while generating a response."

    #     return parsed_response.content

In [9]:
class RAGChatBot:
    def __init__(self, retriever, openai_api_key):
        self.retriever = retriever
        os.environ["OPENAI_API_KEY"] = openai_api_key
        self.chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
        self.prompt_template = hub.pull("rlm/rag-prompt")

    def format_docs(self, docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def generate_response(self, user_query):
        # Retrieve documents relevant to the user query
        relevant_docs = self.retriever.retrieve_documents(user_query)
        relevant_context = self.format_docs(relevant_docs)
        # for relevant_doc in relevant_docs:
            # print(relevant_doc.metadata['source'])
            # print(relevant_doc.page_content)

        user_query = user_query + " Is the patient appropriate for inpatient or outpatient surgery? Why?"
#         instructions = """
#         1. You are a highly knowledgeable and empathetic medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
#         2. If a user chats about some topic on medical device, but doesn't ask/chat about inpatient or outpatient surgery, gently ask the user if you should help him regarding the inpatient vs outpatient surgery
#         3. If a user asks questions outside the medical domain, gently remind them that your expertise is focused on healthcare and surgical advice.
#         4. Chat with the user using the given context. If the context is not at all relevant to the question asked. Tell your reasoning behind your answer and let the user know that you are not sure of the answer, due to lack of relevent context on your end to answer the question.
#         5. If you feel that the data provided is insufficient to answer the question, gently ask the user about more information.
#   """
        instructions = """1. You are a highly knowledgeable medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context given. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save huge costs if you can correctly identify the case of Outpatient Surgery, so please keep this in mind."""
        # Construct the input data for the chain
        input_data = {
            "context": relevant_context,
            "question": user_query,
            "system": instructions# Directly use the user query here
            # "GUIDELINES": instructions
        }
        
        # Create a prompt using the template
        formatted_prompt = self.prompt_template.format(**input_data)
        
        # Generate a response using the LLM with invoke method
        response = self.chat_model.invoke(formatted_prompt)
        
        return response.content

    def generate_response_no_rag(self, question):
        template = """You are a highly knowledgeable and empathetic medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.

Guidelines:
1. **Focus on Medical Context:** Always center your responses around medical topics, particularly those related to surgical procedures and patient care settings.
2. **Inquire When Necessary:** If you require more information to make an informed decision, politely ask the user specific questions about the surgery details, such as the type of surgery, patient health status, or any existing medical conditions.
3. **Educational Approach:** Explain your suggestions clearly, using simple language that both healthcare professionals and patients can understand. Provide insights into the benefits and considerations of each option.
4. **Boundary Setting:** If a user asks questions outside the medical domain, gently remind them that your expertise is focused on healthcare and surgical advice.

Example Interaction:
- **User Query:** "I need to know if my knee surgery should be inpatient or outpatient."
- **Response:** "Knee surgeries can often be performed as outpatient procedures if the patient is in good health and the surgery is minimally invasive. However, if there are underlying health concerns or if a more complex procedure is needed, an inpatient setting might be safer. Could you provide more details about your specific situation?"

By adhering to these guidelines, you ensure that every interaction is informative, supportive, and within the scope of medical advisement.
        """
        template1 = """1. You are a highly knowledgeable medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context you have. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save huge costs if you can correctly identify the case of Outpatient Surgery, so please keep this in mind."""
        system_message_prompt = SystemMessagePromptTemplate.from_template(template1)
        human_template = "{text}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        msg = self.chat_model.invoke(
            chat_prompt.format_prompt(
                text=question,
            ).to_messages()
        )

        return msg.content



    def generate_answer_with_chat_context(self, messages):
        """
        Generates a response with the last few messages and retrieved context using the chat model.

        Args:
            messages (list): List of dictionaries representing the conversation history.

        Returns:
            str: The generated response.
        """
        # Extract the last 4 messages plus the current one
        recent_messages = messages[-5:]  # Last 5 messages (4 historical + current)

        # Prepare input for retrieval (use only the content of messages, excluding the current one)
        retrieval_input = "\n".join(
            f"{message['role'].capitalize()}: {message['content']}" for message in recent_messages[:-1]
        )

        # Attempt to retrieve external documents
        try:
            retrieved_docs = self.retriever.retrieve_documents(retrieval_input)
            retrieved_context = self.format_docs(retrieved_docs)
        except Exception as e:
            retrieved_context = "No additional information could be retrieved."
            print(f"Error during document retrieval: {e}")

        # Ensure retrieved context is a valid string
        retrieved_context = str(retrieved_context)

        # Add a system message with the retrieved context
        system_message_prompt = """1. You are a highly knowledgeable medical chat assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context you have. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save HUGE costs if you can correctly identify the case of Outpatient Surgery, so need not be conservative if you are confident, with the context you have.
        3. If the question/chat is outside the medical domain, gently remind your expertise."""
        system_message = SystemMessage(content= system_message_prompt + f"\n **RELEVANT INFORMATION RETREIVED**:\n{retrieved_context}")
        # system_message = SystemMessage(content="You are a Medical QA chat assistant and you are specialized in making decisions between Inpatient vs Outpatient surgeries.")

        # Convert recent conversation into LangChain message objects
        formatted_recent_messages = []
        for message in recent_messages:
            if message["role"] == "user":
                formatted_recent_messages.append(HumanMessage(content=message["content"]))
            elif message["role"] == "ai":
                formatted_recent_messages.append(AIMessage(content=message["content"]))
            else:
                raise ValueError(f"Unexpected role: {message['role']}")

        # Combine system message and formatted recent messages
        input_messages = [system_message] + formatted_recent_messages
        #formatted_recent_messages.append(HumanMessage(content="Outpatient Surgery saves a lot of money. So never be conservative until otherwise inpatient surgery is ABSOULETELY necessary."))
        # input_messages =  formatted_recent_messages
        # print(input_messages)

        # Generate the response using the chat model
        try:
            llm_response = self.chat_model.invoke(input_messages)
        except Exception as e:
            print(f"Error during model generation: {e}")
            raise

        # Parse the response if needed
        parsed_response = StrOutputParser().parse(llm_response)

        return parsed_response.content


In [10]:
PROMPT_TEMPLATE_NEW = """You are an assistant for medical question-answering tasks. Use the following pieces of retrieved context to answer the question. Read the question with atmost care and follow the instructions given. Use three sentences maximum and keep the answer concise. And include the terms in the context while reasoning.

QUESTION: {question} 

CONTEXT: {context} 

INSTRUCTIONS: {instructions}

ANSWER:"""

"""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:"""

"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\nQuestion: {question} \n\nContext: {context} \n\nAnswer:"

In [11]:
SYSTEM_PROMPT_TEMPLATE_CHAT_HISTORY = """1. You are an assistant for medical question-answering tasks. You are given retrieved CONTEXT and INSTRUCTIONS in this System message, and the chat history attached. Use the following pieces of retrieved CONTEXT to answer the question in the chat and follow the INSTRUCTIONS given. Use three sentences maximum and keep the answer concise and include the terms in the context while reasoning.

CONTEXT: {context} 

INSTRUCTIONS: {instructions}

"""

COMMON_INSTRUCTIONS = """1. You are a highly knowledgeable medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context given. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save huge costs if you can correctly identify the case of Outpatient Surgery, so please keep this in mind. Suggest inpatient surgery only if the given context suggests that the patients are not suitable for outpatient surgery.
        3. If the patients are healthy currently and comorbidities are well controlled or have medical clearence, it is a good sign for Outpatient surgery.
        4. If you do not know the answer from the context you have, Please convey to user that you are not sure.
        5. If the question is not in your domain, please remind user about your expertise."""

In [24]:
class RAGChatBotNew:
    def __init__(self, retriever, openai_api_key):
        self.retriever = retriever
        os.environ["OPENAI_API_KEY"] = openai_api_key
        self.chat_model = ChatOpenAI(model_name="gpt-4o", temperature=0)
        # self.prompt_template = hub.pull("rlm/rag-prompt")
        self.prompt_template = PROMPT_TEMPLATE_NEW
        self.prompt_template_for_chat = SYSTEM_PROMPT_TEMPLATE_CHAT_HISTORY
    def format_docs(self, docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def generate_response(self, user_query):
        # Retrieve documents relevant to the user query
        relevant_docs = self.retriever.retrieve_documents(user_query)
        relevant_context = self.format_docs(relevant_docs)
        print(relevant_context)
        # for relevant_doc in relevant_docs:
            # print(relevant_doc.metadata['source'])
            # print(relevant_doc.page_content)

        user_query = user_query 
#         instructions = """
#         1. You are a highly knowledgeable and empathetic medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
#         2. If a user chats about some topic on medical device, but doesn't ask/chat about inpatient or outpatient surgery, gently ask the user if you should help him regarding the inpatient vs outpatient surgery
#         3. If a user asks questions outside the medical domain, gently remind them that your expertise is focused on healthcare and surgical advice.
#         4. Chat with the user using the given context. If the context is not at all relevant to the question asked. Tell your reasoning behind your answer and let the user know that you are not sure of the answer, due to lack of relevent context on your end to answer the question.
#         5. If you feel that the data provided is insufficient to answer the question, gently ask the user about more information.
#   """
        instructions = COMMON_INSTRUCTIONS
        # Construct the input data for the chain
        input_data = {
            "context": relevant_context,
            "question": user_query,
            "instructions": instructions# Directly use the user query here
            # "GUIDELINES": instructions
        }
        
        # Create a prompt using the template
        formatted_prompt = self.prompt_template.format(**input_data)
        # print(formatted_prompt)
        
        # Generate a response using the LLM with invoke method
        response = self.chat_model.invoke(formatted_prompt)
        
        return response.content

    def generate_response_no_rag(self, question):
        template = """You are a highly knowledgeable and empathetic medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.

Guidelines:
1. **Focus on Medical Context:** Always center your responses around medical topics, particularly those related to surgical procedures and patient care settings.
2. **Inquire When Necessary:** If you require more information to make an informed decision, politely ask the user specific questions about the surgery details, such as the type of surgery, patient health status, or any existing medical conditions.
3. **Educational Approach:** Explain your suggestions clearly, using simple language that both healthcare professionals and patients can understand. Provide insights into the benefits and considerations of each option.
4. **Boundary Setting:** If a user asks questions outside the medical domain, gently remind them that your expertise is focused on healthcare and surgical advice.

Example Interaction:
- **User Query:** "I need to know if my knee surgery should be inpatient or outpatient."
- **Response:** "Knee surgeries can often be performed as outpatient procedures if the patient is in good health and the surgery is minimally invasive. However, if there are underlying health concerns or if a more complex procedure is needed, an inpatient setting might be safer. Could you provide more details about your specific situation?"

By adhering to these guidelines, you ensure that every interaction is informative, supportive, and within the scope of medical advisement.
        """
        template1 = """1. You are a highly knowledgeable medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context you have. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save huge costs if you can correctly identify the case of Outpatient Surgery, so please keep this in mind."""
        system_message_prompt = SystemMessagePromptTemplate.from_template(template1)
        human_template = "{text}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        msg = self.chat_model.invoke(
            chat_prompt.format_prompt(
                text=question,
            ).to_messages()
        )

        return msg.content



    def generate_answer_with_chat_context(self, messages):
        """
        Generates a response with the last few messages and retrieved context using the chat model.

        Args:
            messages (list): List of dictionaries representing the conversation history.

        Returns:
            str: The generated response.
        """
        # Extract the last 4 messages plus the current one
        recent_messages = []
        if len(messages)>5:
            recent_messages = [messages[0]] + [messages[1]]
        recent_messages = recent_messages+ messages[-5:]  # Last 5 messages (4 historical + current)

        # Prepare input for retrieval (use only the content of messages, excluding the current one)
        retrieval_input = "\n".join(
            f"{message['role'].capitalize()}: {message['content']}" for message in recent_messages[:-1]
        )

        # Attempt to retrieve external documents
        try:
            retrieved_docs = self.retriever.retrieve_documents(retrieval_input)
            retrieved_context = self.format_docs(retrieved_docs)
        except Exception as e:
            retrieved_context = "No additional information could be retrieved."
            print(f"Error during document retrieval: {e}")

        # Ensure retrieved context is a valid string
        retrieved_context = str(retrieved_context)

        # Add a system message with the retrieved context
        input_data = {
            "context": retrieved_context,
            "instructions": COMMON_INSTRUCTIONS
        }
        system_message_prompt = self.prompt_template_for_chat.format(**input_data)
        system_message = SystemMessage(content= system_message_prompt)
        # system_message = SystemMessage(content="You are a Medical QA chat assistant and you are specialized in making decisions between Inpatient vs Outpatient surgeries.")

        # Convert recent conversation into LangChain message objects
        formatted_recent_messages = []
        for message in recent_messages:
            if message["role"] == "user":
                formatted_recent_messages.append(HumanMessage(content=message["content"]))
            elif message["role"] == "ai":
                formatted_recent_messages.append(AIMessage(content=message["content"]))
            else:
                raise ValueError(f"Unexpected role: {message['role']}")

        # Combine system message and formatted recent messages
        input_messages = [system_message] + formatted_recent_messages
        #formatted_recent_messages.append(HumanMessage(content="Outpatient Surgery saves a lot of money. So never be conservative until otherwise inpatient surgery is ABSOULETELY necessary."))
        # input_messages =  formatted_recent_messages
        print(formatted_recent_messages)

        # Generate the response using the chat model
        try:
            llm_response = self.chat_model.invoke(input_messages)
        except Exception as e:
            print(f"Error during model generation: {e}")
            raise

        # Parse the response if needed
        parsed_response = StrOutputParser().parse(llm_response)

        return parsed_response.content

In [13]:
print(chat_bot.generate_answer_with_chat_context(messages))

NameError: name 'chat_bot' is not defined

In [19]:
# Configuration and setup
persist_directory = './chroma_db'
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize retriever and chatbot
retriever = DocumentRetriever(persist_directory=persist_directory)
chat_bot = RAGChatBotNew(retriever=retriever, openai_api_key=openai_api_key)

In [21]:
messages = [
    # System Message (Optional but Recommended)
    # {
    #     "role": "system", 
    #     "content": "You are a helpful medical assistant specializing in surgical procedures and patient care."
    # },
    
    # Initial User Message
    {
        "role": "user", 
        "content": "“Patient is a 51 year old female who is planning to undergo laparoscopic myomectomy for fibroids. He has a history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. She is compliant with her medications. There is a medical clearance note provided by her primary care provider. Help me choose inpatient vs outpatient surgery.”"
    }
    ,
    {
        "role": "ai",
        "content": "Given the patient's history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency, along with the medical clearance provided, outpatient surgery could be considered if her comorbidities are well-managed and stable. However, due to the presence of multiple comorbidities, particularly chronic renal insufficiency, inpatient surgery might be more appropriate to ensure close monitoring and management of potential complications. The decision should be made in consultation with her healthcare team, considering her overall health status and the complexity of the procedure."
    },
    {
        "role": "user",
        "content": "What do you suggest? Inpatient or Outpatient?"
        }
   
]


In [22]:
print(chat_bot.generate_answer_with_chat_context(messages))

[HumanMessage(content='“Patient is a 51 year old female who is planning to undergo laparoscopic myomectomy for fibroids. He has a history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. She is compliant with her medications. There is a medical clearance note provided by her primary care provider. Help me choose inpatient vs outpatient surgery.”', additional_kwargs={}, response_metadata={}), AIMessage(content="Given the patient's history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency, along with the medical clearance provided, outpatient surgery could be considered if her comorbidities are well-managed and stable. However, due to the presence of multiple comorbidities, particularly chronic renal insufficiency, inpatient surgery might be more appropriate to ensure close monitoring and management of potential complications. The decision should be made in consultation with her healthcare team, considering her overall health 

In [18]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a helpful assistant! Your name is Bob."
    ),
    HumanMessage(
        content="What is your name?"
    )
]

# Instantiate a chat model and invoke it with the messages
model = ChatOpenAI(model_name="gpt-3.5-turbo")
print(model.invoke(messages))

content='Hello! My name is Bob. How can I assist you today?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 27, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-810bb112-af42-4fdf-9013-b0c41c084376-0'


Question 1

In [17]:
user_query = """Patient is a 94 year old man who is scheduled for laparoscopic cholecystectomy. His BMI is 34. He has a history of end-stage renal disease on intermittent dialysis who receives it every Monday, Wednesday, and Saturday. In addition, he has a history of well-controlled hypertension and insulin-dependent diabetes.Should his surgery be inpatient or outpatient?"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

NameError: name 'chat_bot' is not defined

In [ ]:
user_query = """Patient is a 94 year old man who is scheduled for laparoscopic cholecystectomy. His BMI is 34. He has a history of end-stage renal disease on intermittent dialysis who receives it every Monday, Wednesday, and Saturday. In addition, he has a history of well-controlled hypertension and insulin-dependent diabetes.Should his surgery be inpatient or outpatient?"""
response = chat_bot.generate_response(user_query)
print(response)

The patient should have inpatient surgery. The context indicates that patients on dialysis, like this patient with end-stage renal disease, have significantly increased odds for perioperative mortality and higher complication rates, making them unsuitable for outpatient procedures. Additionally, the patient's advanced age and multiple comorbidities further support the need for inpatient care to ensure close monitoring and management.


In [ ]:
user_query = """What is mathematics?"""
response = chat_bot.generate_response(user_query)
print(response)

I am a medical assistant specializing in advising on surgical procedures, and the question about mathematics is outside my domain of expertise.


Question 2


In [95]:
user_query = """Patient is a 54 year old man who is scheduled for laparoscopic total hysterectomy. Her BMI is 63 and has a history of obstructive sleep apnea but not compliant with CPAP. She had a recent stroke 6 months ago and now on clopidogrel. In addition, she has a history of hypertension and hyperlipidemia, in which she takes amlodipine and a statin."""
user_query = user_query + "Is the patient appropriate for inpatient or outpatient surgery?"
response = chat_bot.generate_response_no_rag(user_query)
print(response)

In this scenario, the patient is a 54-year-old individual with a high BMI of 63, a history of obstructive sleep apnea, recent stroke, and is on clopidogrel, along with hypertension and hyperlipidemia. Given these factors, the patient is more appropriate for inpatient surgery. Here's why:

1. **High BMI**: A BMI of 63 indicates morbid obesity, which increases the risk of complications during and after surgery, such as respiratory issues, wound healing problems, and cardiovascular stress.

2. **Obstructive Sleep Apnea**: The patient's non-compliance with CPAP therapy further elevates the risk of perioperative respiratory complications, especially under anesthesia.

3. **Recent Stroke**: Having had a stroke 6 months ago places the patient at a higher risk for perioperative cardiovascular events. Close monitoring in an inpatient setting is advisable.

4. **Anticoagulation Therapy**: The use of clopidogrel, an antiplatelet medication, increases the risk of bleeding during and after surgery.

In [109]:
user_query = """Patient is a 54 year old man who is scheduled for laparoscopic total hysterectomy. Her BMI is 63 and has a history of obstructive sleep apnea but not compliant with CPAP. She had a recent stroke 6 months ago and now on clopidogrel. In addition, she has a history of hypertension and hyperlipidemia, in which she takes amlodipine and a statin."""
user_query = user_query + " Is the patient appropriate for inpatient or outpatient surgery?"
response = chat_bot.generate_response(user_query)

print(response)

You are an assistant for medical question-answering tasks. Use the following pieces of retrieved context to answer the question. Read the question with atmost care and follow the instructions given. Use three sentences maximum and keep the answer concise. And include the terms in the context while reasoning.

Question: Patient is a 54 year old man who is scheduled for laparoscopic total hysterectomy. Her BMI is 63 and has a history of obstructive sleep apnea but not compliant with CPAP. She had a recent stroke 6 months ago and now on clopidogrel. In addition, she has a history of hypertension and hyperlipidemia, in which she takes amlodipine and a statin. Is the patient appropriate for inpatient or outpatient surgery? 

Context: should consider the invasiveness of the procedure, presence and severity of comorbidities, anesthetic
technique, postoperative opioid requirements, and the surgical venue (e.g., free-standing ASC or
hospital-based outpatient department) should the need for adva

In [62]:
user_query = """Patient is a 26 year old female who is scheduled for an arthroscopic rotator cuff repair. As a child, he had congenital heart disease, which was described as a ventral septal defect in his heart. This was repaired in childhood. Today, he is an active soccer player and has not had any recent issues with his cardiac problems."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

In determining whether the arthroscopic rotator cuff repair should be conducted as an inpatient or outpatient procedure, several factors need to be considered:

1. **Type of Surgery**: Arthroscopic rotator cuff repair is commonly performed as an outpatient procedure. It is minimally invasive, typically involves less postoperative pain, and has a quicker recovery time compared to open surgery.

2. **Patient's Cardiac History**: The patient had a ventral septal defect repaired in childhood and has not experienced any recent cardiac issues. This suggests that the previous heart condition is well-managed and not currently impacting the patient's health. However, it is crucial to ensure that the patient's cardiac status is stable and that there are no residual effects that could complicate anesthesia or recovery.

3. **Current Health and Activity Level**: The patient is an active soccer player, indicating a good level of physical fitness and cardiovascular health. This is a positive factor 

In [116]:
user_query = """Patient is a 26 year old female who is scheduled for an arthroscopic rotator cuff repair. As a child, he had congenital heart disease, which was described as a ventral septal defect in his heart. This was repaired in childhood. Today, he is an active soccer player and has not had any recent issues with his cardiac problems."""
user_query = user_query + "Is the patient appropriate for inpatient or outpatient surgery?"
response = chat_bot.generate_response(user_query)
print(response)

The patient, a 26-year-old female with a history of congenital heart disease that was repaired in childhood and who is currently an active soccer player without recent cardiac issues, appears to be a suitable candidate for outpatient surgery. The context suggests that if a patient's comorbidities are well controlled and they are healthy, outpatient surgery is appropriate. Given her current health status and lack of recent cardiac problems, outpatient surgery for the arthroscopic rotator cuff repair seems appropriate.


In [106]:
user_query = "What are the instructions given to you?"
response = chat_bot.generate_response(user_query)
print(response)

You are a highly knowledgeable medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery should be conducted as an inpatient or outpatient procedure, based on the context provided. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.


In [39]:
user_query = """Patient is a 83 year old man who is scheduled for a ureteral stent placement for kidney stones. His BMI is 42. He has a history of well-controlled COPD, hypertension, obstructive sleep apnea on CPAP, and coronary artery disease. He had a myocardial infarction 5 years ago, in which he underwent a CABG. Since then, he has not had any episodes of myocardial ischemia."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Given the patient's medical history and current health status, the decision between an inpatient or outpatient ureteral stent placement procedure is crucial. Let's break down the considerations:

1. **Ureteral Stent Placement Procedure:** This procedure involves placing a small tube (stent) in the ureter to help urine flow from the kidney to the bladder. It is commonly done to treat kidney stones or other urinary obstructions.

2. **Patient's Age and BMI:** The patient is 83 years old with a BMI of 42, which may increase the risk of complications during and after the procedure, such as wound healing issues and infections.

3. **Medical History:** The patient has a history of COPD, hypertension, obstructive sleep apnea, and coronary artery disease with a previous CABG. These conditions, especially COPD and coronary artery disease, may impact the patient's ability to tolerate the procedure and anesthesia.

4. **Myocardial Infarction History:** The patient had a myocardial infarction 5 ye

In [115]:
user_query = """Patient is a 83 year old man who is scheduled for a ureteral stent placement for kidney stones. His BMI is 42. He has a history of well-controlled COPD, hypertension, obstructive sleep apnea on CPAP, and coronary artery disease. He had a myocardial infarction 5 years ago, in which he underwent a CABG. Since then, he has not had any episodes of myocardial ischemia."""
user_query = user_query + "Is the patient appropriate for inpatient or outpatient surgery?"
response = chat_bot.generate_response(user_query)
print(response)

The patient has a high BMI of 42 and a history of coronary artery disease with a past myocardial infarction, which are significant comorbidities. Although his conditions like COPD, hypertension, and obstructive sleep apnea are well-controlled, the presence of coronary artery disease and prior cardiac surgery are independent predictors of longer length of stay and mortality, suggesting a higher risk for outpatient surgery. Therefore, inpatient surgery is more appropriate for this patient to ensure careful monitoring and management of potential complications.


In [41]:
user_query = """Patient is a 72 year old female who is scheduled for total shoulder arthroplasty. She has a history of ischemic cardiomyopathy and hypertension. Her last echo demonstrated an ejection of fraction of 25% with severe mitral stenosis. She is unable to walk up 2 flights of stairs without feeling short of breath."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Based on the patient's medical history and current health status, it is recommended that the total shoulder arthroplasty be conducted as an inpatient procedure. Here's why:

1. **Complexity of the Surgery:** Total shoulder arthroplasty is a significant surgical procedure that involves replacing the damaged parts of the shoulder joint with artificial components. Given the patient's age and medical conditions, including ischemic cardiomyopathy and severe mitral stenosis, there may be an increased risk of complications during and after the surgery.

2. **Cardiac Health Concerns:** The patient's history of ischemic cardiomyopathy, hypertension, and a low ejection fraction of 25% indicate a compromised heart function. An inpatient setting allows for closer monitoring of the patient's cardiac status throughout the surgery and during the initial recovery period.

3. **Functional Limitations:** The patient's difficulty in walking up stairs without feeling short of breath suggests limited funct

In [117]:
user_query = """Patient is a 72 year old female who is scheduled for total shoulder arthroplasty. She has a history of ischemic cardiomyopathy and hypertension. Her last echo demonstrated an ejection of fraction of 25% with severe mitral stenosis. She is unable to walk up 2 flights of stairs without feeling short of breath."""
user_query = user_query + " Is the patient appropriate for inpatient/main hospital or oupatient surgery?"
response = chat_bot.generate_response(user_query)
print(response)

The patient is not appropriate for outpatient surgery due to her severe mitral stenosis and symptomatic presentation, including dyspnea and poor exercise tolerance. The context specifies that patients with severe valve disease and symptoms such as dyspnea are not suitable candidates for ambulatory surgery. Therefore, inpatient surgery is recommended for this patient.


In [48]:
user_query = """Patient is a 84 year old male with an ASA classification score of 4, who is scheduled for a laparoscopic inguinal hernia repair. He has smoked 100 pack years and has severe COPD requiring continuous oxygen. In addition, he has end-stage renal disease and is awaiting dialysis placement."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Based on the information provided, the patient is an 84-year-old male with multiple significant health issues, including a high ASA classification score of 4, severe COPD, end-stage renal disease, and a history of heavy smoking. Given these complex medical conditions, the decision between an inpatient or outpatient setting for the laparoscopic inguinal hernia repair is crucial.

In this case, considering the patient's age, ASA score, severe COPD requiring continuous oxygen, and end-stage renal disease, it is highly recommended that the laparoscopic inguinal hernia repair be performed as an inpatient procedure. Here's why:

1. **Monitoring and Management:** In an inpatient setting, the patient can be closely monitored post-surgery for any potential complications that may arise due to his multiple health issues. Continuous oxygen support and specialized care for COPD and renal disease can be provided effectively in a hospital environment.

2. **Risk of Complications:** The patient's hist

In [118]:
user_query = """Patient is a 84 year old male with an ASA classification score of 4, who is scheduled for a laparoscopic inguinal hernia repair. He has smoked 100 pack years and has severe COPD requiring continuous oxygen. In addition, he has end-stage renal disease and is awaiting dialysis placement."""
response = chat_bot.generate_response(user_query)
print(response)

The patient, with an ASA classification score of 4, severe COPD requiring continuous oxygen, and end-stage renal disease, is at high risk for adverse outcomes after ambulatory surgery. COPD is identified as an independent risk factor, and patients with severely diminished respiratory function are not good candidates for outpatient surgery. Given these factors, inpatient surgery is recommended to ensure appropriate monitoring and management of potential complications.


In [50]:
user_query ="""Patient is a 63 year old man who is planning to undergo total knee arthroplasty. He had a history of multiple episodes of ventricular fibrillation and eventually underwent placement of a AICD one year ago. He is unable to walk up two flights of stairs without feeling chest pain. In addition, he has a history of diabetes, hypertension, and obstructive sleep apnea.
"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Given the patient's complex medical history, including a previous history of ventricular fibrillation, placement of an Automatic Implantable Cardioverter Defibrillator (AICD), chest pain with exertion, diabetes, hypertension, and obstructive sleep apnea, it is recommended that the total knee arthroplasty be performed as an inpatient procedure.

Here's why:
1. **Cardiac History:** The patient's history of ventricular fibrillation and AICD placement indicates a significant cardiac condition. An inpatient setting allows for closer monitoring of the patient's cardiac status during and after the surgery.
   
2. **Chest Pain:** The chest pain experienced by the patient with exertion could be related to underlying cardiac issues. In an inpatient setting, immediate access to cardiac monitoring and interventions is available if needed.

3. **Multiple Comorbidities:** The presence of diabetes, hypertension, and obstructive sleep apnea increases the risk of complications during and after surgery.

In [82]:
user_query ="""Patient is a 63 year old man who is planning to undergo total knee arthroplasty. He had a history of multiple episodes of ventricular fibrillation and eventually underwent placement of a AICD one year ago. He is unable to walk up two flights of stairs without feeling chest pain. In addition, he has a history of diabetes, hypertension, and obstructive sleep apnea.
"""
response = chat_bot.generate_response(user_query)
print(response)

The patient is more appropriate for inpatient surgery due to his history of major cardiac issues, including multiple episodes of ventricular fibrillation and the presence of an AICD, as well as his inability to walk up two flights of stairs without chest pain. The context suggests that patients with major cardiac or pulmonary disease are excluded from outpatient arthroplasty. Additionally, his history of diabetes, hypertension, and obstructive sleep apnea further supports the need for inpatient care to manage potential complications.


In [19]:
user_query ="""Patient is a 51 year old female who is planning to undergo laparoscopic myomectomy for fibroids. He has a history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. She is compliant with her medications. There is a medical clearance note provided by her primary care provider.
"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Based on the provided information, the patient's medical history includes hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. It is important to consider these comorbidities when determining whether the laparoscopic myomectomy for fibroids should be conducted as an inpatient or outpatient procedure.

In this case, due to the patient's medical history of multiple comorbidities, including hypertension, lupus, asthma, and chronic renal insufficiency, it is advisable to recommend that the laparoscopic myomectomy be conducted as an inpatient procedure rather than an outpatient procedure. 

Here are the reasons for this recommendation:

1. **Medical History**: The patient's medical history includes chronic conditions such as hypertension, lupus, asthma, and chronic renal insufficiency. These conditions may increase the risk of complications during and after surgery. Being in an inpatient setting allows for closer monitoring of the patient's condition post-surgery, e

In [120]:
user_query ="""Patient is a 51 year old female who is planning to undergo laparoscopic myomectomy for fibroids. She has a history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. She is compliant with her medications. There is a medical clearance note provided by her primary care provider.
"""
response = chat_bot.generate_response(user_query)
print(response)

The patient, a 51-year-old female with a history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency, has received medical clearance from her primary care provider, which is a positive indicator for outpatient surgery. While chronic renal insufficiency is a concern, the context suggests that with well-managed comorbidities and medical clearance, outpatient laparoscopic myomectomy could be considered. However, careful patient selection and optimization of her conditions are crucial to ensure safety in the ambulatory setting.


In [54]:
user_query ="""Patient is a 64 year old man who is planning to undergo an umbilical hernia repair. He has a history of ischemic cardiomyopathy (with recent ejection fraction of 30%), cirrhosis with persistent ascites, end-stage renal disease on dialysis, moderate pulmonary hypertension, and moderate aortic stenosis. He had a myocardial infarction 5 months, in which is received a drug eluting stent to his LAD. He is now on dual anti-platelet therapy.
"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Given the complex medical history of the patient, it is advisable for the umbilical hernia repair to be performed as an inpatient procedure rather than outpatient. Here's why:

1. **Multiple Comorbidities:** The patient's history of ischemic cardiomyopathy, cirrhosis, end-stage renal disease, pulmonary hypertension, aortic stenosis, and recent myocardial infarction with a drug-eluting stent places him at high risk for complications during and after surgery. These conditions may require close monitoring and specialized care that are more readily available in an inpatient setting.

2. **Medication Management:** The patient is currently on dual anti-platelet therapy following the placement of a drug-eluting stent. In an inpatient setting, healthcare providers can closely monitor and manage the patient's medications to prevent any interactions or adverse effects during and after the surgery.

3. **Postoperative Care:** Given the patient's compromised health status, he may require more inte

In [14]:
user_query ="""Patient is a 64 year old man who is planning to undergo an umbilical hernia repair. He has a history of ischemic cardiomyopathy (with recent ejection fraction of 30%), cirrhosis with persistent ascites, end-stage renal disease on dialysis, moderate pulmonary hypertension, and moderate aortic stenosis. He had a myocardial infarction 5 months, in which is received a drug eluting stent to his LAD. He is now on dual anti-platelet therapy.
"""
response = chat_bot.generate_response(user_query)
print(response)

The patient is not appropriate for outpatient surgery due to his comorbidities, including moderate aortic stenosis and recent myocardial infarction with a drug-eluting stent. Patients with severe valve disease and recent myocardial infarction are excluded from outpatient surgery unless specific criteria are met. The patient's complex medical history suggests that inpatient surgery would be more appropriate for his safety and care.


In [18]:
user_query ="""Patient is a 98 year old female who is planning to undergo total thyroidectomy. He has a BMI of 27 and has a history of hypertension, diabetes, hyperlipidemia, and depression. His last hemoglobin A1c was 7.2.
"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Based on the information provided, it is recommended that the total thyroidectomy be conducted as an inpatient procedure rather than an outpatient procedure for the following reasons:

1. Age: The patient is 98 years old, which may increase the risk of complications during and after surgery. Older patients tend to have weaker immune systems and may take longer to recover from surgical procedures. Being in an inpatient setting allows for closer monitoring and immediate intervention if any complications arise.

2. Medical History: The patient has a history of hypertension, diabetes, hyperlipidemia, and depression. These comorbidities can increase the risk of surgical complications, such as poor wound healing, infections, and cardiovascular events. In an inpatient setting, the patient can receive specialized care and monitoring to manage these conditions effectively before, during, and after surgery.

3. BMI and Hemoglobin A1c: The patient has a BMI of 27 and a hemoglobin A1c of 7.2, indi

In [122]:
user_query ="""Patient is a 98 year old female who is planning to undergo total thyroidectomy. He has a BMI of 27 and has a history of hypertension, diabetes, hyperlipidemia, and depression(but he is healthy now without them). His last hemoglobin A1c was 7.2.
"""
user_query = user_query + "Is the patient appropriate for inpatient or outpatient surgery?"
response = chat_bot.generate_response(user_query)
print(response)

The patient, a 98-year-old female with a history of hypertension, diabetes, hyperlipidemia, and depression, has a hemoglobin A1c of 7.2, indicating moderate glycemic control. The context suggests that outpatient surgery is generally appropriate for patients with good long-term blood glucose control unless there are significant complications of hyperglycemia, which are not indicated here. Therefore, based on the provided context and the patient's current health status, outpatient surgery could be considered appropriate.


In [121]:
user_query ="My BMI is 27.0. Let me know if you need more info."
response = chat_bot.generate_response(user_query)
print(response)

Based on the context provided, a BMI of 27.0 falls within the overweight category, but BMI alone should not be the sole determining factor for surgical procedure selection. It is important to consider other factors such as the type of surgical procedure, comorbidities, and functional capacity. Since the context does not provide specific information about these other factors, I am not sure whether outpatient surgery is appropriate in this case.


In [ ]:
user_query = """Patient is a 84 year old male with an ASA classification score of 4, who is scheduled for a laparoscopic inguinal hernia repair. He has smoked 100 pack years and has severe COPD requiring continuous oxygen. In addition, he has end-stage renal disease and is awaiting dialysis placement."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Based on the information provided, the patient is an 84-year-old male with multiple significant health issues, including a high ASA classification score of 4, severe COPD, end-stage renal disease, and a history of heavy smoking. Given these complex medical conditions, the decision between an inpatient or outpatient setting for the laparoscopic inguinal hernia repair is crucial.

In this case, considering the patient's age, ASA score, severe COPD requiring continuous oxygen, and end-stage renal disease, it is highly recommended that the laparoscopic inguinal hernia repair be performed as an inpatient procedure. Here's why:

1. **Monitoring and Management:** In an inpatient setting, the patient can be closely monitored post-surgery for any potential complications that may arise due to his multiple health issues. Continuous oxygen support and specialized care for COPD and renal disease can be provided effectively in a hospital environment.

2. **Risk of Complications:** The patient's hist